In [1]:
#uncomment to clean up the variables
%reset -f

In [2]:
import sys
import os
import matplotlib.pyplot as plt
from astropy import constants
import skycalc_ipy
from mpdaf.obj import Spectrum
#from astropy import constants

import warnings
warnings.filterwarnings('ignore')

# Add the project root directory to Python path
project_root = os.path.dirname(os.getcwd())  # Adjust path as needed
sys.path.insert(0, project_root)

from pyetc_dev import *

#this is a package of mine for plot quality, comment it and the following row in case you use this notebook 
#from setplot import set_plot_style
#set_plot_style()

In [3]:
### set up the container
#obj = WST(log = 'DEBUG', skip_dataload = False)
obj = VLT(skip_dataload = False)

In [4]:
# read the fits file and store it in a dictionary
from astropy.io import fits
import pandas as pd
from astropy.table import Table


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

data_dict = {}

# read the fits file and store it in a dictionary
tab = Table.read('/Users/andre/Desktop/INAF/WST/template.fits', format='fits')
data_dict = tab.to_pandas()

In [5]:
#SPEC = template, pl, bb, line

#SP_BIN = binning of the spectral trace, integer

#TEMP_NAME is the name of the template, you can list them with sed_models.eso_spectra_files.keys()

################ only for line #############################
#SPEC_range = fixed, adaptative, None

#### only for fixed ####
#SPEC_HSIZE = 5 [range will be 2 * SPEC_HSIZE + 1]
########################

#### only for adaptative ####
#SPEC_KFWHM = 3 [range will be +- SPEC_KFWHM]
#OPT_SPEC = True, to compute the best SPEC_KFWHM for the maximization of the SNR
#############################
############################################################


#OBJ = sb, ps, resolved

################ only for resolved ###############################
#IMA = moffat, sersic 

#IMA_ELL = 0.

#### only for moffat ####
#IMA_FWHM = 1. in arcsec
#IMA_BETA = 2.5 in arcsec
#IMA_KFWHM = 3 by default
#########################

#### only for sersic ####
#IMA_REFF = 1. in arcsec
#IMA_N = 2.
#IMA_KREFF = 6 by default
#########################

##################################################################

################ only for sb ###############################
#IMA_AREA = 1. arcsec, area to use for the SNR computation
############################################################

################ only for ps and resolved ##################
#IMA_RANGE = circular_adaptative, square_fixed

#### only for square_fixed ####
#IMA_HSIZE = 5 [range will be 2 * IMA_HSIZE + 1] #INTEGER
###############################

#### only for circular_adaptative ####
#IMA_KFWHM = 3 [range will be +- IMA_KFWHM]
#OPT_IMA = True, to compute the best IMA_KFWHM for the maximization of the SNR
######################################
############################################################

### "SKYCALC": if false it uses the default configurations in the folder using MOON, 
###true it computes the sky with a call to the ESO Skycalc and want the FLI and PWV parameters

# SNR & SNR_WAVE are the target SNR at a specific wave if you want to compute the NDIT or DIT

#Here an example of how it should be


full_obs = {
    "INS": "ifs",
    "CH": "red",
    
    "NDIT": 1,
    "DIT": 1800, 
    
    "SNR": 80,
    "Lam_Ref":5500,
    
    "OBJ_FIB_DISP": 0,
    
    "MOON": 'greysky',
    "PWV": 10,
    "MFLI": 1,
    "SEE": 0.8,
    "AM": 1.16,
    "SKYCALC": True,
    
    "Obj_SED": 'bb',
    "SED_Name": 'Pickles_K7V',
    
    "OBJ_MAG": 18,
    "MAG_SYS": 'Vega',
    "MAG_FIL": 'R',
    
    "Z": 0,
    "BB_Temp": 9000.,
    "PL_Index": None,
    
    "SEL_FLUX": 50e-16,
    "SEL_CWAV": 6000,
    "SEL_FWHM":-2,
    
    "Obj_Spat_Dis": 'ps',
    
    "IMA": 'moffat',
    "Ext_Ell": 0.,
    
    "IMA_FWHM": 1,
    "IMA_BETA": 999,
    "IMA_KFWHM": 5,
    
    "Sersic_Reff": 8.00,
    "Sersic_Ind": 4.00,
    "IMA_KREFF": 5,
    
    "SPEC_RANGE": 'fixed',
    "SPEC_KFWHM": 10,
    "SPEC_HSIZE": int(3601/2),
    
    "COADD_WL": 2,
    
    "IMA_RANGE": 'square_fixed',
    "COADD_XY": 3,
    
    "OPT_SPEC": False,
    "OPT_IMA": False,
    "FRAC_SPEC_MEAN_OPT_IMAGE": 1
}

Obj_SED = {
    1: 'pl',
    2: 'bb',
    3: 'pl',
    4: 'template',
    5: 'line'
}

# Object Spatial Distribution
Obj_Spat_Dis = {
    1: 'ps',
    3: 'sb',
    4: 'sersic'
}

filters = {
   1: 'U',
   2: 'B',
   3: 'V',
   4: 'R',
   5: 'I',
   6: 'J',
   7: 'H',
   8: 'K',
   9: 'uSDSS',
   10: 'gSDSS',
   11: 'rSDSS',
   12: 'iSDSS',
   13: 'zSDSS',
   14: 'jSDSS',
   15: 'uLSST',
   16: 'gLSST',
   17: 'rLSST',
   18: 'iLSST',
   19: 'zLSST',
}

MAG_SYS = {
    1: 'AB',
    2: 'Vega'
}

INS = {
    1: 'ifs',
    2: 'MOS'
}

CH = {
    1: 'blue',
    2: 'red'
}


In [10]:
res_dict = data_dict.copy()

# iter over the res_dict row by row
for index, row in res_dict.iterrows():
    print("Processing row: ", row['ID'])

    for key in row.keys():
        full_obs[key] = row[key]
    
    # SED
    try:
        full_obs['Obj_SED'] = Obj_SED[row['Obj_SED']]
        if row['Obj_SED'] == 1:
            full_obs['PL_Index'] = 0
        if row['Obj_SED'] == 4:
            full_obs['SED_Name'] = row['SED_Name'].decode('utf-8')
    except KeyError as e:
        print("Error in SED", e)
        res_dict.loc[index, 'ERR'] = 1
        continue

    # Object Spatial Distribution
    try:
        full_obs['Obj_Spat_Dis'] = Obj_Spat_Dis[row['Obj_Spat_Dis']]
    except KeyError as e:
        print("Error in Object Spatial Distribution", e)
        res_dict.loc[index, 'ERR'] = 1
        continue

    # filters
    try:
        if pd.isna(row['MAG_FIL']):
            full_obs['MAG_FIL'] = None
        else:
            full_obs['MAG_FIL'] = filters[row['MAG_FIL']]
    except KeyError as e:
        print("Error in MAG_FIL", e)
        res_dict.loc[index, 'ERR'] = 1
        continue

    # MAG_SYS
    try:
        if pd.isna(row['MAG_SYS']):
            full_obs['MAG_SYS'] = None
        else:
            full_obs['MAG_SYS'] = MAG_SYS[row['MAG_SYS']]
    except KeyError as e:
        print("Error in MAG_SYS", e)
        res_dict.loc[index, 'ERR'] = 1
        continue

    # INS 
    try:
        full_obs['INS'] = INS[row['INS']]
    except KeyError as e:
        print("Error in INS", e)
        res_dict.loc[index, 'ERR'] = 1
        continue

    # CH
    try:
        full_obs['CH'] = CH[row['CH']]
    except KeyError as e:
        print("Error in CH", e)
        res_dict.loc[index, 'ERR'] = 1
        continue

    # get DIT
    DIT = full_obs['DIT']
    SNR = full_obs['SNR']

    con, ob, spe, im, spe_input = obj.build_obs_full(full_obs)

    if np.isnan(DIT):
        res = obj.time_from_source(con, im, spe, dit=True)
        # DEBUG ****************************************************
        print("*** SNR check ***")
        full_obs_snr = full_obs.copy()
        full_obs_snr['SNR'] = None
        full_obs_snr['DIT'] = res['dit']
        con, ob, spe, im, spe_input = obj.build_obs_full(full_obs_snr)
        if full_obs['COADD_WL'] > 1:
            print('SNR check: ' + str(obj.snr_from_source(con, im, spe)['spec']['snr_rebin'].subspec(full_obs['Lam_Ref'], unit=u.angstrom)))
        else:
            print('SNR check: ' + str(obj.snr_from_source(con, im, spe)['spec']['snr'].subspec(full_obs['Lam_Ref'], unit=u.angstrom)))
        # DEBUG ****************************************************
    else:
        if np.isnan(SNR):
            if full_obs['COADD_WL'] > 1:
                res = {'SNR':obj.snr_from_source(con, im, spe)['spec']['snr_rebin'].subspec(full_obs['Lam_Ref'], unit=u.angstrom)}
            else:
                res = {'SNR':obj.snr_from_source(con, im, spe)['spec']['snr'].subspec(full_obs['Lam_Ref'], unit=u.angstrom)}
        else:
            res = obj.time_from_source(con, im, spe, dit=False)
            # DEBUG ****************************************************
            print("*** SNR check ***")
            full_obs_snr = full_obs.copy()
            full_obs_snr['SNR'] = None
            full_obs_snr['NDIT'] = res['ndit']
            con, ob, spe, im, spe_input = obj.build_obs_full(full_obs_snr)
            if full_obs['COADD_WL'] > 1:
                print('SNR check: ' + str(obj.snr_from_source(con, im, spe)['spec']['snr_rebin'].subspec(full_obs['Lam_Ref'], unit=u.angstrom)))
            else:
                print('SNR check: ' + str(obj.snr_from_source(con, im, spe)['spec']['snr'].subspec(full_obs['Lam_Ref'], unit=u.angstrom)))
            # DEBUG ****************************************************
        
    for key in res.keys():
        # sey key uppercase
        dict_key = key.upper()
        res_dict.loc[index, dict_key] = res[key]

    print("DONE WITH ROW: ", row['ID'])

print(res_dict)
        



Processing row:  1
Computing DIT for target SNR: 80.0
Computed DIT: 2555.52 > 2556.00 for target SNR: 80.00 at wavelength 7000.00 A, in 9 spaxels
% % % Computing fraction of saturated pixels % % %
Fraction of saturated voxels: 0.0% for saturation level 65000.00 counts
Time elapsed: 0.01 seconds
*** SNR check ***
Time elapsed: 0.35 seconds
SNR check: 79.66209919344189
DONE WITH ROW:  1
Processing row:  2
Computing DIT for target SNR: 20.0
Computed DIT: 2210.13 > 2211.00 for target SNR: 20.00 at wavelength 7000.00 A, in 9 spaxels
% % % Computing fraction of saturated pixels % % %
Fraction of saturated voxels: 0.0% for saturation level 65000.00 counts
Time elapsed: 0.01 seconds
*** SNR check ***
Time elapsed: 0.28 seconds
SNR check: 20.00371787934336
DONE WITH ROW:  2
Processing row:  3
Computing NDIT for target SNR: 20.0
Computed NDIT: 4.17 > 5.00 for target SNR: 20.00 at wavelength 7000.00 A, in 9 spaxels
% % % Computing fraction of saturated pixels % % %
Fraction of saturated voxels: 0